# MNIST Digit Recognition
## Introduction
This function of this Jupyter notebook is to explain how the python script [Source link to script](https://github.com/kbarry91/Emerging-Technologies/tree/master/4-MNIST%20Digit%20Recognition%20Script) works and to discuss its performance.
<br>

The aim of the script as discribed in the project brief was to develope a __Digit recognition script__: a _Python script that takes an image ﬁle containing a handwritten digit and identiﬁes the digit using a supervised learning algorithm and the MNIST dataset._


## Folder Structure 
On inital download of the project you will notice the directory only has one folder 'testImages' and a 'digitrec.py' script
<br>
__testImages:__ Contain images of different sizes and types to test the algorithm.<br>
__digitrec.py:__ A python script containing the source code
![initial](https://i.imgur.com/8nkerH3.png)
Once the program has been compiled and run another folder 'models' will be generated. 
![after](https://i.imgur.com/LTGgkJI.png)
__models:__ Contains saved neural network model to increase performace and decrease wait time 

## Navigating The program
The program has been build to provide simplicity to the user. That is it hides all the complicated workings from the user with a command line GUI that is simple to navigate.

### The main Menu 
The main menu allows the user to view a brief summary of the amount of images used to build and test the model. Aswell as this it also gives 4 options. An option is selected by entering a input into the command line
<br>
__1.Prepare Dataset and Image Recognition Model:__ Allows user to Build the Digit Recognition model.<br>
__2.Upload an Image to predict:__ Allows user to enter an image for the _testImages_ folder and make a prediction.<br>
__3.Use MNIST image to predict:__ Allows user to select an image from the MNIST training set to make a prediction.<br>
__4.Exit:__ Exits the program
![Menu](https://i.imgur.com/1MtWXvs.png)

### Prepare Dataset and Image Recognition model
Select option _1_ allows the user to Prepare the Dataset and to create the image recognition model. The user will be prompted with the option to select how many iterations will be used to train the model.Once the model is build it is saved in the directory as _nn_model_.
<br>
The program will then display the following statistics
1. Summary info, The layers and shape of the model. (Achieved by the code _'model.summary()'_)
2. Build progress of the model training  (model.fit(image_train, label_train,
                                            batch_size=batch_size,
                                            epochs=epochs,
                                            verbose=1,
                                            validation_data=(image_test, label_test))
3. Build complete summary info including loss, accuracy and time taken

![build1](https://i.imgur.com/UwQ4DRS.png)
![build2](https://i.imgur.com/JEJ0wNK.png)

### Upload an Image to predict
Selecting the second menu option (2) allows the user to enter an image and run a predction. Any image that is located in the _testImages_ folder can be used. Once the prediction is complete the program will output in descending order the accuracy percentage of each possible outcome along with a brief note of what the recognised image is.
![imgUplaod](https://i.imgur.com/0m11ETt.png)

### Use MNIST image to predict
Selcting the third menu option (3) allows the user to harness the scope of the MNIST dataset. The MNIST test set contains 10000 images which are loaded to the program ayt runtime. Selecting a value of 1-10000 will run a prediction on the specified image and output the result.
![mnistImg](https://i.imgur.com/m7AIQ2J.png)

# How The program works
As the program is is quite large I have __removed some functionality and comments for the purpose of keeping it short__ and explaining how it works. 

The program has numerous menu options that set variables according to a user input.In this snippet all menu __options have been replaced with hardcoded values.__
To view the full commented code view [Source link to script](https://github.com/kbarry91/Emerging-Technologies/tree/master/4-MNIST%20Digit%20Recognition%20Script)

### Imports used


In [1]:
import gzip          # Must import gzip to allow python to read  and uncompress zip files
import timeit        # Timer
import numpy as np  # Import numpy as np
import sys
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop

import cv2# Import Cv2  and Imagefor image processing
from PIL import Image

from keras.models import load_model # Import load_models to save model
# Import os path to read file
import os.path

Using TensorFlow backend.


### Prepare The Data
The data from the MNIST data set must be loaded into memory before it can be used. Once loaded the data must be reshaped too 60000/10000 items of size 784(28*28). We must also convert the set to a binary representation so it can be used to calculate categorical crossentropy loss on the model.

In [2]:
(image_train, label_train), (image_test, label_test) = mnist.load_data()
imageSize = 784
labelOptions = 10

# Reshape image data set to 60000 and 10000 elements of size784
image_train = image_train.reshape(60000, imageSize)
image_test = image_test.reshape(10000, imageSize)

# Convert image data to type float 32
image_train = image_train.astype('float32')
image_test = image_test.astype('float32')

# Values are rgb 0-255 . Convert to  0 OR  1
image_train /= 255
image_test /= 255

# convert class vectors to binary class matrices
# A binary matrix representation of the input. The classes axis is placed last.
# Used for  categorical_crossentropy loss on model
label_train = keras.utils.to_categorical(label_train, labelOptions)
label_test = keras.utils.to_categorical(label_test, labelOptions)
print(label_test)

# Output Statistics
print(image_train.shape[0], 'training images loaded.')
print(image_test.shape[0], 'test images loaded.')
print("Preparing Mnist data Complete!")

[[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
60000 training images loaded.
10000 test images loaded.
Preparing Mnist data Complete!


As you can see above the data has been reshaped and now the training set holds 60,000 images whilst the test set has 10,000.

### Build model
The model must be created and I chose sequential to use a linear stack of layers. We add 3 densely-connected nn-layers, we also set a dropout rate to prevent the model from overfitting while training. For this notebook I have set the epoch value to one to allow one forward pass and one backwards pass of all the training examples. We set the batch size to 128 to set the number of training examples in one forward and backward pass.

In [3]:
epochs = 1
batch_size = 128

# Using Sequental ,Linear stack of layers
# Add layer off input shape 784 and output shape of *532
# Set fraction of input rates to drop during training
model = Sequential()  
model.add(Dense(512, activation='relu', input_shape=(784,))) 
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(labelOptions, activation='softmax'))

# Print a string summary of the model
model.summary()

# Configure model for training
# loss = name of objective function
# metrics list of metrics to be evaluated by the model

# Compile the model
# Using sgd gives .86 accur takinhg 18s
# Using RMSprop() gives .98 taking 23s
# rmsprop = RMSprop(lr=learning_rate) gives .28  taking 25s
rmsprop = RMSprop()
model.compile(loss='categorical_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

# Start timer and build model
print("\nBuilding Model...")
start_time_train = timeit.default_timer()

# Train the model
# verbose 1 displays progress bar
history = model.fit(image_train, label_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(image_test, label_test))

# Stop Timer
print("\nModel built !")
end_time_train = timeit.default_timer() - start_time_train

score = model.evaluate(image_test, label_test, verbose=0)
print('Test loss    :', score[0])
print('Test accuracy:', score[1])
print("Time Taken to train model at ", epochs, "epochs =", end_time_train)



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
_________________________________________________________________

Building Model...
Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 23s 385us/

### Make A Predicition
Now that the model is created we can make a prediction, As you can see from the following output the neural network works as it should and predicts the correct output with an accuracy of nearly 100%.

In [8]:
predictionB = model.predict(np.array([image_test[1]], dtype=float))

# Get value of closest(MAX) prediction
predAccuracy = max(predictionB[0])

# Get index of closest(MAX) prediction/actual
predIndex = predictionB.argmax(axis=1)
actIndex = label_test[1].argmax(axis=0)

# Print Prediction results
print("\nPrediction Complete")

# Output formatted result to console
print("System Predicted image is :",predIndex ,",With a accuracy of ","{0:.4f}%".format(predAccuracy*100) )
print("Actual Image is: ", actIndex)



Prediction Complete
System Predicted image is : [2] ,With a accuracy of  99.9658%
Actual Image is:  2


### Make Prediction

## Performance 

The model performs very well in both runtime and prediction accuracy.

### Optimzer
RMSprop is an unpulished, adaptive learning rate method for reccurent neural networks.for more info see the official keras documentation ![RMSprop](https://keras.io/optimizers/)
I decide to use the __RMSprop()__ optimizer as it gave the best runtime/accuracy when running tests against the test images.

| Optimizer                 | epoch | batch size | accuracy | Runtime |
|---------------------------|-------|------------|----------|---------|
| RMSprop(default)          | 1     | 128        | 99.23%   | 18s     |
| sgd                       | 1     | 128        | 86%      | 23s     |
| RMSprop(lr=learning_rate) | 1     | 128        | 26%      | 25s     |
| Adagrad                   | 1     | 128        | 28%      | 25s     |

### Epoch
I also had to fine tune the epoch value and discovered at a rate of 5 gives the best results. The following table is using the RMSprop.

| epoch | batch size | accuracy | Runtime | loss |
|-------|------------|----------|---------|------|
| 1     | 128        | 96.52%   | 21      | 0.1  |
| 5     | 128        | 99.23%   | 107     | 0.07 |
| 10    | 128        | 98.32%%  | 218     | 0.05 |

## Testing and NN results
The neural network has proven to be very effective. The model has been able to predict most Images with a near 100% accuracy. I have tested the model against an array of different images such as the official MNIST images,png,jpeg,skewed images, images i have created myself and images of different size.
Below are screenshots of some of results outputted by the 

## References
https://keras.io/optimizers/